In [1]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

# Lister

Adapted from [Unit 10: lister.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/lister.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/r.txt).

Associated lecture video: Unit 10 lesson 1

## Problem statement

Sir Joseph Lister (1827--1912), Professor of Surgery at Glasgow University, influenced by Pasteur's ideas, found that a wound wrapped in bandages treated by Carbolic acid (phenol) would often not become infected.

Here are Lister's data on treating open fractures and amputations:

Period    Carbolic acid used    Results

1864--1866     No       Treated   34 patients, 15 died and 19 recovered

1867--1870     Yes     Treated   40 patients,   6 died and 34 recovered


Estimate and interpret the risk difference,  risk ratio, and   odds ratio


## Notes

- Wishart! I've tried both WishartBartlett and LKJ, can't get a similar result to the professor's yet.
- possibly useful paper:
    - https://arxiv.org/pdf/1809.04746.pdf


Unfinished! This is not working correctly right now, assuming because of the different tau prior.

In [4]:
n1 = 34
y1 = 15
n2 = 40
y2 = 6
mu0 = np.array([-0.5, -1.5])
S = np.array([[0.1, 0], [0, 0.1]])

In [13]:
with pm.Model() as m:
    tau = pm.WishartBartlett("tau", S=S, nu=8)
    sigma = pm.Deterministic("sigma", pm.math.matrix_inverse(tau))
    mu = pm.MvNormal("mu", mu=mu0, cov=tau, shape=2)

    p1 = mu[0]
    p2 = mu[1]

    pm.Binomial("y1", logit_p=p1, n=n1, observed=y1)
    pm.Binomial("y2", logit_p=p2, n=n2, observed=y2)

    pm.Deterministic("rd", p1 - p2)
    pm.Deterministic("rr", p1 / p2)
    pm.Deterministic("or", (p1 / (1 - p1)) / (p2 / (1 - p2)))

    trace = pm.sample(3000)

Added new variable tau_c to model diagonal of Wishart.
Added new variable tau_z to model off-diagonals of Wishart.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau_c, tau_z, mu]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 21 seconds.
There were 12 divergences after tuning. Increase `target_accept` or reparameterize.
There were 13 divergences after tuning. Increase `target_accept` or reparameterize.
There were 10 divergences after tuning. Increase `target_accept` or reparameterize.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.


In [14]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
tau_z[0],-0.044,0.960,-1.910,1.683,0.009,0.009,11300.0,8101.0,1.0
mu[0],-0.303,0.308,-0.889,0.272,0.003,0.002,10143.0,7973.0,1.0
mu[1],-1.690,0.365,-2.377,-1.016,0.004,0.003,10761.0,8102.0,1.0
tau_c[0],5.366,3.182,0.482,11.026,0.030,0.021,8755.0,5604.0,1.0
tau_c[1],4.498,2.934,0.301,9.739,0.026,0.019,10816.0,6379.0,1.0
"tau[0, 0]",0.537,0.318,0.048,1.103,0.003,0.002,8755.0,5604.0,1.0
"tau[0, 1]",-0.009,0.223,-0.449,0.414,0.002,0.002,10677.0,8929.0,1.0
"tau[1, 0]",-0.009,0.223,-0.449,0.414,0.002,0.002,10677.0,8929.0,1.0
"tau[1, 1]",0.542,0.322,0.053,1.126,0.003,0.002,9832.0,7299.0,1.0
"sigma[0, 0]",3.749,3.860,0.546,9.484,0.053,0.038,8291.0,6200.0,1.0


In [2]:
%watermark -n -u -v -iv -p aesara,aeppl

Last updated: Sun Jul 31 2022

Python implementation: CPython
Python version       : 3.10.5
IPython version      : 8.4.0

aesara: 2.7.7
aeppl : 0.0.32

arviz: 0.12.1
pymc : 4.1.3
numpy: 1.23.1



or ~ 2.8
rd ~ 0.2
rr ~2.1